<a href="https://colab.research.google.com/github/yeonui-0626/lstm/blob/main/lstm_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
from google.colab import drive
import io
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import concat
from matplotlib import pyplot
from datetime import datetime
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
#import keras.backend.tensorflow_backend as K

In [7]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
#%%time
data = pd.read_csv('/content/gdrive/MyDrive/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv', index_col=2)
data.loc[data['Label']=='Benign','Label'] = 0
data.loc[data['Label']=='Bot','Label'] = 1
data.index.name = 'date'

# 결측치  저리
data = data.replace([np.inf, -np.inf], np.nan)
data['Flow Byts/s'].fillna(0.0,inplace=True)
data['Flow Pkts/s'].fillna(0.0,inplace=True)
values = data.values
values = values.astype('float64')
 
# 정규화
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(values)
label = scaled[:,-1]

# def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
#     n_vars = 1 if type(data) is list else data.shape[1]
#     df = DataFrame(data)
#     cols, names = list(), list()
#     # input sequence (t-n, ... t-1)
#     for i in range(n_in, 0, -1):
#         cols.append(df.shift(i))
#         names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
#     # forecast sequence (t, t+1, ... t+n)
#     for i in range(0, n_out):
#         cols.append(df.shift(-i))
#         if i == 0:
#             names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
#         else:
#             names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
#     # put it all together
#     agg = concat(cols, axis=1)
#     agg.columns = names
#     # drop rows with NaN values
#     if dropnan:
#         agg.dropna(inplace=True)
#     return agg
# timesteps = 1
# reframed = series_to_supervised(scaled,timesteps,1)
# values = reframed.values

timesteps = 1
train_x, test_x, train_y, test_y = train_test_split(scaled, label, test_size=0.3, random_state=1004)

# reshape input to be 3D [samples, timesteps, features]
train_x = train_x.reshape((train_x.shape[0], timesteps, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], timesteps, test_x.shape[1]))

In [ ]:
%%time
print("시작")
# model
#with K.tf.device('/gpu:0'):
timesteps = 1
feature = train_x.shape[2]
model = Sequential()
model.add(LSTM(256, input_shape=(timesteps, feature)))
#model.add(Dropout(0.2))
model.add(Dense(1,activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop')
model.summary()

history = model.fit(train_x,train_y, epochs=10, batch_size=100, validation_data=(test_x, test_y), verbose=1, shuffle=False)
print('\n## training loss and acc ##')
print(history.history['loss'],end='\n\n')

yhat = model.predict(test_x)
print("총 예측수 :",yhat.shape)
print("Bot 탐지수 :",yhat[yhat==1].shape)
print("정상 탐지수 :",yhat[yhat==0].shape)

del [[model, history]]